In [2]:
# =====================================================================
#  C5 verification – “first-zero” termination test
# ---------------------------------------------------------------------
#  Goal
#  ----
#  Numerically confirm empirical conclusion **(C5)**:
#
#      A converged phase-difference trajectory δ(t) stops at the very
#      first zero of f(δ) that lies in the direction indicated by
#      sign [f(δ₀)].
#
#  Input
#  -----
#  • **ode_steady_analysis.xlsx**  – produced by the C1 script.
#      It contains one row per simulated launch with the columns
#          delta0, p1, p2, a, steady_value, match_type, …
#      Only rows whose  match_type ∈ {"(1)", "(2)"}  are kept, meaning
#      the trajectory reached a numerical steady state and f(δ) has at
#      least one zero in the enclosing π-period.
#
#  Operational reformulation implemented here
#  ------------------------------------------
#  For each qualifying launch (δ₀, p₁, p₂, a):
#
#    1.  Evaluate f(δ) on the intervals
#            [δ₀ − π, δ₀]       (left side, 2 000 points)  and
#            [δ₀,     δ₀ + π]   (right side, 2 000 points)
#        and locate every sign change to obtain the zero lists
#            left_zeroes,  right_zeroes.
#
#    2.  Pick the *nearest* zero to the left (Z_L) and to the right
#        (Z_R), if they exist.
#
#    3.  Compare the plateau value δ★ with those nearest zeros using
#        a match tolerance  MATCH_TOLERANCE = 0.01 rad:
#
#            "(l)"   |δ★−Z_L| < tol
#            "(r)"   |δ★−Z_R| < tol
#            "(nl)"  δ★ < δ₀  but δ★ ≠ Z_L          ← C5 violation
#            "(nr)"  δ★ > δ₀  but δ★ ≠ Z_R          ← C5 violation
#            "(?)"   logic gap / should never occur
#
#        C5 is validated when the outcome set is **exactly** {"(l)",
#        "(r)"} – i.e. no “overshoot” or “gap” rows.
#
#  Output
#  ------
#  • Console summary:
#        Total entries checked
#        Satisfying extended zero-match hypothesis ((r)+(l))
#        Breakdown by nearest_zero_match type
#
#  Implementation notes
#  --------------------
#  • The cell–fibre ODE is *not* re-integrated here; we trust C1’s
#    steady state (steady_value) and simply interrogate f(δ).
#  • b is fixed at 1.0 (same as in C1).
#  • MATCH_TOLERANCE = 0.01 ≈ 0.57° gives headroom for numerical error
#    while still detecting genuine overshoots.
# =====================================================================

import numpy as np
import pandas as pd
from math import sin, cos, pi

# === Load original data with match_type labels (1–5) ===
df = pd.read_excel("ode_steady_analysis.xlsx")

# === Define f(Δ) function ===
def f_delta(delta, p1, p2, a):
    b = 1.0
    sin2d = np.sin(2 * delta)
    sin_d = np.sin(delta)
    cos_d = np.cos(delta)
    denom = 2 * ((a * sin_d)**2 + (b * cos_d)**2)**1.5
    dTheta_dt = -1.0 - p1 * ((a * b) * ((a**2 - b**2) * sin2d) / denom)
    dPsi_dt = -p2 * sin2d
    return dPsi_dt - dTheta_dt

# === Parameters ===
MATCH_TOLERANCE = 0.01

# === Only consider cases with steady state and zero crossings (case (1) and (2)) ===
df_sub = df[df["match_type"].isin(["(1)", "(2)"])].reset_index(drop=True)

# === Re-label based on extended hypothesis: left/right nearest zero matching ===
new_labels = []

for _, row in df_sub.iterrows():
    delta0 = row["delta0"]
    p1 = row["p1"]
    p2 = row["p2"]
    a = row["a"]
    steady = row["steady_value"]

    # Compute f(Δ) over extended range
    left_range = np.linspace(delta0 - pi, delta0, 2000)
    right_range = np.linspace(delta0, delta0 + pi, 2000)

    f_left = f_delta(left_range, p1, p2, a)
    f_right = f_delta(right_range, p1, p2, a)

    # Find left-side zeroes
    left_zeroes = []
    for i in range(len(f_left) - 1):
        if f_left[i] * f_left[i + 1] < 0:
            zero = (left_range[i] + left_range[i + 1]) / 2
            left_zeroes.append(zero)

    # Find right-side zeroes
    right_zeroes = []
    for i in range(len(f_right) - 1):
        if f_right[i] * f_right[i + 1] < 0:
            zero = (right_range[i] + right_range[i + 1]) / 2
            right_zeroes.append(zero)

    # Identify nearest left/right zero if any
    nearest_left = None
    nearest_right = None

    if left_zeroes:
        left_zeroes = np.array(left_zeroes)
        nearest_left = left_zeroes[np.argmin(np.abs(left_zeroes - delta0))]
    if right_zeroes:
        right_zeroes = np.array(right_zeroes)
        nearest_right = right_zeroes[np.argmin(np.abs(right_zeroes - delta0))]

    # === Matching logic ===
    if nearest_right is not None and abs(steady - nearest_right) < MATCH_TOLERANCE:
        new_labels.append("(r)")  # matches right nearest
    elif nearest_left is not None and abs(steady - nearest_left) < MATCH_TOLERANCE:
        new_labels.append("(l)")  # matches left nearest
    elif nearest_left is not None and steady < delta0:
        new_labels.append("(nl)")  # doesn't match left, but lies on left
    elif nearest_right is not None and steady > delta0:
        new_labels.append("(nr)")  # doesn't match right, but lies on right
    else:
        new_labels.append("(?)")  # catch-all, should not occur if logic is exhaustive

# === Add new label column ===
df_sub["nearest_zero_match"] = new_labels

# === Statistics ===
counts = df_sub["nearest_zero_match"].value_counts().sort_index()
total = len(df_sub)
satisfied = counts.get("(r)", 0) + counts.get("(l)", 0)
satisfying_ratio = satisfied / total

# === Output results ===
print(f"Total entries checked: {total}")
print(f"Satisfying extended zero-match hypothesis ((r)+(l)): {satisfied} ({satisfying_ratio:.2%})\n")
print("Breakdown by nearest_zero_match type:")
print(counts.to_string())


Total entries checked: 7560
Satisfying extended zero-match hypothesis ((r)+(l)): 7560 (100.00%)

Breakdown by nearest_zero_match type:
nearest_zero_match
(l)    1192
(r)    6368
